<a href="https://colab.research.google.com/github/gnoziere/cs230-ak-troll-detector/blob/main/Troll_Detector_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load data from CSV files

In [133]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [134]:
!pip3 install transformers tweet-preprocessor emoji

In [135]:
import math
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from transformers import AutoModel, AutoTokenizer

In [136]:
device = "cuda"

In [137]:
# Define paths
positive_data_path = "/content/drive/MyDrive/turkey_052020_tweets_csv_hashed_2011.csv"
negative_data_path = "/content/drive/MyDrive/turkey_052020_tweets_csv_hashed_2020_01.csv"
model_path = "dbmdz/distilbert-base-turkish-cased"

In [138]:
import os
cwd = os.getcwd()

#positive_raw_data = pd.read_csv(cwd+'/testing.csv', error_bad_lines=False)
positive_raw_data = pd.read_csv(positive_data_path)
positive_raw_data["label"] = 1.0

#negative_raw_data = pd.read_csv(cwd+'/turkey_052020_tweets_csv_hashed_2020_01.csv', nrows=1000 ,low_memory=False, error_bad_lines=False)
negative_raw_data = pd.read_csv(negative_data_path)
negative_raw_data["label"] = 0.0

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [139]:
positive_raw_data.columns

Index(['tweetid', 'userid', 'user_display_name', 'user_screen_name',
       'user_reported_location', 'user_profile_description',
       'user_profile_url', 'follower_count', 'following_count',
       'account_creation_date', 'account_language', 'tweet_language',
       'tweet_text', 'tweet_time', 'tweet_client_name', 'in_reply_to_userid',
       'in_reply_to_tweetid', 'quoted_tweet_tweetid', 'is_retweet',
       'retweet_userid', 'retweet_tweetid', 'latitude', 'longitude',
       'quote_count', 'reply_count', 'like_count', 'retweet_count', 'hashtags',
       'urls', 'user_mentions', 'label'],
      dtype='object')

In [140]:
positive_raw_data.shape

(35100, 31)

In [141]:
negative_raw_data.shape

(859516, 31)

In [142]:
merged_data = pd.concat([positive_raw_data, negative_raw_data], ignore_index=True)
merged_data = merged_data.sample(frac=1)

In [143]:
merged_data.shape

(894616, 31)

# Pre-process data

In [144]:
merged_data = merged_data.drop([
    'latitude', # always 'absent' in positive dataset
    'longitude',
    'user_profile_url', # always 'NaN in positive dataset
    'in_reply_to_userid',
    'in_reply_to_tweetid',
    'quoted_tweet_tweetid',
    'retweet_userid',
    'retweet_tweetid',
    'quote_count', # can’t access these without a premium developer account
    'reply_count' 
], axis = 1)

In [145]:
merged_data.columns

Index(['tweetid', 'userid', 'user_display_name', 'user_screen_name',
       'user_reported_location', 'user_profile_description', 'follower_count',
       'following_count', 'account_creation_date', 'account_language',
       'tweet_language', 'tweet_text', 'tweet_time', 'tweet_client_name',
       'is_retweet', 'like_count', 'retweet_count', 'hashtags', 'urls',
       'user_mentions', 'label'],
      dtype='object')

In [146]:
merged_data.shape

(894616, 21)

In [147]:
# remove [,], ', NaN from hashtags, urls, and user_mentions
merged_data['hashtags'] = merged_data['hashtags'].str.replace('[','').str.replace(']','').str.replace("'",'').str.lower()
merged_data['urls'] = merged_data['urls'].str.replace('[','').str.replace(']','').str.replace("'",'')
merged_data['user_mentions'] = merged_data['user_mentions'].str.replace('[','').str.replace(']','').str.replace("'",'')

merged_data = merged_data.fillna('')

In [148]:
import preprocessor as p
import emoji
import string

# remove emojis  
def remove_emoji(text):
    return emoji.get_emoji_regexp().sub(u'', text)

# remove url, smiley, hashtag, mention and reserved
p.set_options(p.OPT.URL, p.OPT.SMILEY, p.OPT.HASHTAG, p.OPT.MENTION, p.OPT.RESERVED)

for i in merged_data.index:
  text = merged_data.at[i, 'tweet_text']
  text = p.clean(str(text))
  text = remove_emoji(text)
  text = text.lower().replace('[^\w\s]',' ').replace('\s\s+', ' ').replace('\r', ' ').replace('\n', ' ').translate(str.maketrans('', '', string.punctuation));
  merged_data.at[i, 'tweet_text'] = text

  text = merged_data.at[i, 'user_display_name']
  text = p.clean(str(text))
  text = remove_emoji(text)
  text = text.lower().replace('[^\w\s]',' ').replace('\s\s+', ' ').replace('\r', ' ').replace('\n', ' ').translate(str.maketrans('', '', string.punctuation));
  merged_data.at[i, 'user_display_name'] = text

  text = merged_data.at[i, 'user_profile_description']
  text = p.clean(str(text))
  text = remove_emoji(text)
  text = text.lower().replace('[^\w\s]',' ').replace('\s\s+', ' ').replace('\r', ' ').replace('\n', ' ').translate(str.maketrans('', '', string.punctuation));
  merged_data.at[i, 'user_profile_description'] = text

In [149]:
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
merged_data.tweet_text

846170    bizler ve mahkumlarımız cok zor durumda hele b...
545822     i̇tibarınıza şatafatınıza varaklı eşyalarınız...
402417     meclisin görevi haksızlığa vurguna talana dur...
713903     i̇slahiye’de muhtarlarımızı topladı muhtarlar...
93172      esnaf demek türki̇ye demekti̇r… esnaf ile dol...
                                ...                        
343841                     wi kuzey ege haber  wi başkanı …
269815     biz isimsizleriz biz kimsesizlerin kimsesiyiz...
791977    dydvh7ebvssqo bana ağırlık ve hazine lazım değ...
234798    konkordato alana ceza yok alamayana 5 yil hapi...
595153                  rahmet ve saygıyla makamı âli olsun
Name: tweet_text, Length: 894616, dtype: object

In [150]:
merged_data.shape

(894616, 21)

In [151]:
today = pd.Timestamp('2021-11-30')
merged_data['account_creation_date'] = pd.to_datetime(merged_data.account_creation_date)
merged_data['account_age'] = (today - merged_data.account_creation_date).dt.days

In [152]:
merged_data.account_age

846170     983
545822     895
402417     904
713903     895
93172      769
          ... 
343841     880
269815    2022
791977    2471
234798    3613
595153     847
Name: account_age, Length: 894616, dtype: int64

In [153]:
merged_data['is_turkish_tweet'] = merged_data.tweet_language == 'tr'
merged_data['is_turkish_account'] = merged_data.account_language == 'tr'
merged_data['is_android'] = merged_data.tweet_client_name == 'Twitter for Android'
merged_data['is_iphone'] = merged_data.tweet_client_name == 'Twitter for iPhone'
merged_data['is_mobile'] = merged_data.is_android & merged_data.is_iphone
merged_data['contains_hashtag'] = len(merged_data.hashtags) > 0
merged_data['contains_url'] = len(merged_data.urls) > 0

In [154]:
merged_data[:10]

,tweetid,userid,user_display_name,user_screen_name,user_reported_location,user_profile_description,follower_count,following_count,account_creation_date,account_language,tweet_language,tweet_text,tweet_time,tweet_client_name,is_retweet,like_count,retweet_count,hashtags,urls,user_mentions,label,account_age,is_turkish_tweet,is_turkish_account,is_android,is_iphone,is_mobile,contains_hashtag,contains_url
846170,1214287856220422144,du9lnDIQ9b1V5VOYxTRBhvN+eQeGs0+TOM993oHMi8=,du9lndiq9b1v5voyxtrbhvneqegs0tom993ohmi8,du9lnDIQ9b1V5VOYxTRBhvN+eQeGs0+TOM993oHMi8=,", bu vatan bizim",,488,200,2019-03-23,tr,tr,bizler ve mahkumlarımız cok zor durumda hele b...,2020-01-06 20:49,Twitter Web Client,False,0,0,,,3147501676,0.0,983,True,True,False,False,False,True,True
545822,1220788330763255810,pcFZ7NxBN3OgtA067v5AsOHQTuzMIwc98q6sXT5N3lg=,pcfz7nxbn3ogta067v5asohqtuzmiwc98q6sxt5n3lg,pcFZ7NxBN3OgtA067v5AsOHQTuzMIwc98q6sXT5N3lg=,Turkey,,2459,761,2019-06-19,en,tr,i̇tibarınıza şatafatınıza varaklı eşyalarınız...,2020-01-24 19:19,Twitter for Android,True,0,0,,,1036333596448096258,0.0,895,True,False,True,False,False,True,True
402417,1217874053786152960,cgAoXf5VjXh29IaDKAJMYdYvquzW6rS4MO6tX1HF54=,cgaoxf5vjxh29iadkajmydyvquzw6rs4mo6tx1hf54,cgAoXf5VjXh29IaDKAJMYdYvquzW6rS4MO6tX1HF54=,Türkiye,,1967,2005,2019-06-10,en,tr,meclisin görevi haksızlığa vurguna talana dur...,2020-01-16 18:19,Twitter for iPhone,True,0,0,,,1138719651708514304,0.0,904,True,False,False,True,False,True,True
713903,1219936815878152193,1141259785133858816,ℂ✯mehmet rei̇s ℂ✯,vali1453,,r҉e҉i҉s҉ t҉i҉m҉ g҉r҉u҉p҉l҉a҉r҉i҉ g҉r҉u҉p҉ b҉a҉...,6048,5932,2019-06-19,tr,tr,i̇slahiye’de muhtarlarımızı topladı muhtarlar...,2020-01-22 10:56,Twitter for Android,True,0,0,,,"201218863, 778121595927793665",0.0,895,True,True,True,False,False,True,True
93172,1213184224607555585,LJNX4QCcUMHEPA12PCayXIzvr6M7rRnrkTV6BtD58n8=,ljnx4qccumhepa12pcayxizvr6m7rrnrktv6btd58n8,LJNX4QCcUMHEPA12PCayXIzvr6M7rRnrkTV6BtD58n8=,,eşi ödeyemediği çeklerden dolayı cezaevine gir...,350,642,2019-10-23,en,tr,esnaf demek türki̇ye demekti̇r… esnaf ile dol...,2020-01-03 19:43,Twitter for iPhone,True,0,0,,,1126909028179816453,0.0,769,True,False,False,True,False,True,True
470935,1220106358474801153,GZx+ovoBhQr1geozCHIc2M+L2pXdnmXJzkwvce9Zmes=,gzxovobhqr1geozchic2ml2pxdnmxjzkwvce9zmes,GZx+ovoBhQr1geozCHIc2M+L2pXdnmXJzkwvce9Zmes=,,recep durak,309,793,2019-10-21,en,tr,ödenemeyen çekler 240 artti ağustos 21978 ad ...,2020-01-22 22:09,Twitter for iPhone,True,0,0,,,1218986762174980097,0.0,771,True,False,False,True,False,True,True
261836,1221413947552780288,IG+lkSBHbaVflHrJQIA3C4Sj9bvWD2ACVVc1Jz0v9Zs=,iglksbhbavflhrjqia3c4sj9bvwd2acvvc1jz0v9zs,IG+lkSBHbaVflHrJQIA3C4Sj9bvWD2ACVVc1Jz0v9Zs=,,pek anlatılcak bir tarafı yok hayatımın yaşadı...,1256,1771,2018-10-21,tr,tr,68’lik deprem ile içimiz yandı ölülerimiz ve ...,2020-01-26 12:45,Twitter for Android,True,0,0,elaziğ,,845184812,0.0,1136,True,True,True,False,False,True,True
805060,1219721165436063744,BC8akkAe9WGza5aXB2iB5gqtTBvqldZLTg8JNG1VSD0=,bc8akkae9wgza5axb2ib5gqttbvqldzltg8jng1vsd0,BC8akkAe9WGza5aXB2iB5gqtTBvqldZLTg8JNG1VSD0=,,gruplarımıza katılmak ister misiniz iletişim,854,709,2018-05-22,tr,tr,dünyanın en güzel ritmi bir insanın rabbi iç...,2020-01-21 20:39,Twitter for Android,True,0,0,,,1200107004469170177,0.0,1288,True,True,True,False,False,True,True
524344,1220453952359227407,3059267103,uin,BaskanTR71,Türkiye,atasını tanımayan it peşinde gezer,9280,3461,2015-02-24,tr,tr,kemalistlerin atatürkü istismar ettiği gibi b...,2020-01-23 21:11,Twitter for Android,True,0,0,,,1122510174105763841,0.0,2471,True,True,True,False,False,True,True
475797,1213515097890598915,1141259785133858816,ℂ✯mehmet rei̇s ℂ✯,vali1453,,r҉e҉i҉s҉ t҉i҉m҉ g҉r҉u҉p҉l҉a҉r҉i҉ g҉r҉u҉p҉ b҉a҉...,6048,5932,2019-06-19,tr,tr,allahim ben her halimi sana bıraktım sen beni...,2020-01-04 17:38,Twitter for Android,True,0,0,,,1186365112854159362,0.0,895,True,True,True,False,False,True,True


# Split sets

In [155]:
def split_data(data, train=0.98, test=0.01, eval=0.01):
  assert train + test + eval == 1.0

  num_examples = data.shape[0]
  data = data.sample(frac=1)

  train_data = data[:int(num_examples * train)]
  test_data = data[int(num_examples * train):int(num_examples * (train+test))]
  eval_data = data[int(num_examples * (train+test)):]  # sacred data

  return (train_data, test_data, eval_data)

In [156]:
train_data, test_data, eval_data = split_data(merged_data, train=0.8, test=0.1, eval=0.1)

In [157]:
train_data.shape

(715692, 29)

In [158]:
test_data.shape

(89462, 29)

In [159]:
eval_data.shape

(89462, 29)

In [160]:
train_data.columns  # ground truth columns

Index(['tweetid', 'userid', 'user_display_name', 'user_screen_name',
       'user_reported_location', 'user_profile_description', 'follower_count',
       'following_count', 'account_creation_date', 'account_language',
       'tweet_language', 'tweet_text', 'tweet_time', 'tweet_client_name',
       'is_retweet', 'like_count', 'retweet_count', 'hashtags', 'urls',
       'user_mentions', 'label', 'account_age', 'is_turkish_tweet',
       'is_turkish_account', 'is_android', 'is_iphone', 'is_mobile',
       'contains_hashtag', 'contains_url'],
      dtype='object')

# Define features

In [166]:
# Feature info
float_features_names = [
  "follower_count",
  "following_count",
  "like_count",
  "retweet_count",
  "account_age",
  "is_retweet",  # binary
  "is_mobile",  # binary
  "is_turkish_tweet",  # binary
  "is_turkish_account",  # binary
  "contains_hashtag",  # binary
  "contains_url",  # binary
]

In [167]:
# Data pre-processing
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Define model steps

In [168]:
def process_data(batch):
  feature_dict = tokenizer(
      batch["tweet_text"].tolist(),
      padding=True,
      truncation=True,
      return_tensors="pt",
  )

  numerical_data = batch[float_features_names].astype(float)
  feature_dict["float_features"] = torch.from_numpy(numerical_data.values).float()

  labels = batch["label"].astype(float)
  feature_dict["labels"] = torch.from_numpy(labels.values).float()

  return feature_dict

In [169]:
class BertTweetClassifier(nn.Module):
  def __init__(self, bert_size, bert_red_size, numeric_feature_size, red_size, dropout=0.1):
    super().__init__()
    self.bert_size = bert_size

    # Use pre-trained BERT model
    self.bert = AutoModel.from_pretrained(
        model_path,
        output_hidden_states=True,
        output_attentions=True,
    )

    for param in self.bert.parameters():
        param.requires_grad = False  # No backprop here for now

    self.weights = nn.Parameter(torch.rand(7, 1))
    self.dropout = nn.Dropout(dropout)
    self.fc1 = nn.Linear(bert_size, bert_red_size)
    self.fc2 = nn.Linear(bert_red_size + numeric_feature_size, red_size)
    self.fc3 = nn.Linear(red_size, 1)  # binary classfication
    self.relu = nn.ReLU()
    self.sigmoid = nn.Sigmoid()

  def forward(self, feature_dict):
    input_ids = feature_dict["input_ids"]
    float_features = feature_dict["float_features"]

    all_hidden_states, all_attentions = self.bert(input_ids, output_attentions=True, output_hidden_states=True)[-2:]
    batch_size = input_ids.shape[0]

    ht_cls = torch.cat(all_hidden_states)[:, :1, :].view(7, batch_size, 1, self.bert_size)
    atten = torch.sum(ht_cls * self.weights.view(7, 1, 1, 1), dim=[1, 3])

    atten = F.softmax(atten.view(-1), dim=0)
    feature = torch.sum(ht_cls * atten.view(7, 1, 1, 1), dim=[0, 2])

    bert_red_out = self.fc1(self.dropout(feature))
    bert_red_activ_out = self.relu(bert_red_out)

    concat_layer = torch.cat((bert_red_activ_out, float_features), 1)
    concat_out = self.fc2(concat_layer)
    concat_activ_out = self.relu(concat_out)

    out = self.fc3(concat_activ_out)
    prediction = self.sigmoid(out)

    return prediction

In [170]:
model = BertTweetClassifier(768, 64, len(float_features_names), 32).to(device)

Some weights of the model checkpoint at dbmdz/distilbert-base-turkish-cased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [171]:
model

BertTweetClassifier(
  (bert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in

# Train model

In [172]:
import time
import gc
gc.collect()

2288

In [180]:
def report_eval_metrics(model, test_data, step=0.01, batch_size=256):
  counts = {}
  metrics = {}
  num_examples = test_data.shape[0]

  torch.cuda.empty_cache()
  model.zero_grad()

  for threshold in np.arange(0.0, 1.0 + step, step):
    counts[threshold] = np.zeros(4)  # TP, TN, FP, FN in that order
    metrics[threshold] = np.zeros(5)  # accuracy, precision, recall, specificity, f1_score

  with torch.no_grad():
    for batch_start_index in range(0, num_examples, batch_size):
      batch_num = int(batch_start_index / batch_size)
      if batch_num % 25 == 0:
        print(f"Batch {batch_num} started")

      batch_data = test_data[batch_num : min(batch_num + batch_size, num_examples)]

      feature_dict = process_data(batch_data).to(device)
      predictions = np.squeeze(model(feature_dict))
      labels = feature_dict["labels"]

      for threshold in np.arange(0.0, 1.0 + step, step):
        binary_predictions = (predictions > threshold).float()
        
        counts[threshold][0] += torch.bitwise_and(binary_predictions == 1.0, labels == 1.0).sum()
        counts[threshold][1] += torch.bitwise_and(binary_predictions == 0.0, labels == 0.0).sum()
        counts[threshold][2] += torch.bitwise_and(binary_predictions == 1.0, labels == 0.0).sum()
        counts[threshold][3] += torch.bitwise_and(binary_predictions == 0.0, labels == 1.0).sum()

      del predictions
      torch.cuda.empty_cache()

  for threshold in np.arange(0.0, 1.0 + step, step):
    metrics[threshold][0] = ((counts[threshold][0] + counts[threshold][1]) / num_examples).item()
    metrics[threshold][1] = (counts[threshold][0] / (counts[threshold][0] + counts[threshold][2])).item()
    metrics[threshold][2] = (counts[threshold][0] / (counts[threshold][0] + counts[threshold][3])).item()
    metrics[threshold][3] = (counts[threshold][1] / (counts[threshold][2] + counts[threshold][1])).item() 
    metrics[threshold][4] = ((2*counts[threshold][0]) / (2*counts[threshold][0] + counts[threshold][2] + counts[threshold][3])).item()

  return metrics

In [183]:
pre_training_metrics = report_eval_metrics(model, test_data, step=0.01)

Batch 0 started
Batch 25 started
Batch 50 started
Batch 75 started
Batch 100 started
Batch 125 started
Batch 150 started
Batch 175 started
Batch 200 started
Batch 225 started
Batch 250 started
Batch 275 started
Batch 300 started
Batch 325 started


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars


In [186]:
pre_training_metrics

{0.0: array([0.04047529, 0.04041295, 1.        , 0.        , 0.07768636]),
 0.01: array([0.08462811, 0.04227671, 1.        , 0.04594145, 0.08112377]),
 0.02: array([0.0949677 , 0.04273827, 1.        , 0.05669989, 0.08197315]),
 0.03: array([0.09665556, 0.04281458, 1.        , 0.05845613, 0.0821135 ]),
 0.04: array([0.09801927, 0.04287643, 1.        , 0.05987509, 0.08222724]),
 0.05: array([0.09801927, 0.04287643, 1.        , 0.05987509, 0.08222724]),
 0.06: array([0.09849992, 0.04289827, 1.        , 0.06037521, 0.08226741]),
 0.07: array([0.09994187, 0.04296393, 1.        , 0.06187557, 0.08238814]),
 0.08: array([0.100892  , 0.0430073 , 1.        , 0.06286419, 0.08246789]),
 0.09: array([0.10114909, 0.04301906, 1.        , 0.06313169, 0.08248949]),
 0.1: array([0.10812412, 0.04334036, 1.        , 0.07038928, 0.08307999]),
 0.11: array([0.10872773, 0.04336839, 1.        , 0.07101734, 0.08313149]),
 0.12: array([0.11480852, 0.0436528 , 1.        , 0.07734447, 0.08365388]),
 0.13: array([

In [199]:
model.fc1.weight

Parameter containing:
tensor([[-0.0160, -0.0060,  0.0009,  ..., -0.0294,  0.0193,  0.0085],
        [ 0.0169, -0.0224,  0.0194,  ...,  0.0046,  0.0070, -0.0186],
        [ 0.0216, -0.0330, -0.0259,  ..., -0.0335, -0.0104, -0.0100],
        ...,
        [ 0.0021,  0.0123, -0.0095,  ...,  0.0023, -0.0052, -0.0254],
        [ 0.0285,  0.0117,  0.0054,  ...,  0.0232,  0.0092,  0.0262],
        [ 0.0028, -0.0004,  0.0136,  ..., -0.0100,  0.0065,  0.0059]],
       device='cuda:0', requires_grad=True)

In [200]:
batch_size = 64
learning_rate = 1e-5
num_epochs = 3
num_examples = train_data.shape[0]
criterion = nn.BCELoss()

torch.cuda.empty_cache()
model.zero_grad()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

batch_count = math.ceil(num_examples / batch_size)

for epoch_num in range(num_epochs):
  print(f"Epoch {epoch_num} started")
  train_data = train_data.sample(frac=1)  # Shuffle the batches each epoch
  total_epoch_loss = 0

  ### 0.4 seconds total per loop ###
  for batch_start_index in range(0, num_examples, batch_size):
    model.zero_grad()

    batch_num = int(batch_start_index / batch_size)
    if batch_num % 250 == 0:
      print(f"Train batch {batch_num} started")

    batch_data = train_data[batch_num : min(batch_num + batch_size, num_examples)]

    feature_dict = process_data(batch_data).to(device)
    predictions = np.squeeze(model(feature_dict))  ## TODO: Speed up
    labels = feature_dict["labels"]

    loss = criterion(predictions, labels)
    total_epoch_loss += loss.item()

    loss.backward()
    optimizer.step()

    ### 0.1 seconds start ###
    del predictions
    torch.cuda.empty_cache()
    ### 0.1 seconds end ###

  print(f"Epoch {epoch_num} finished")
  print("Average batch loss: " + str(total_epoch_loss / batch_count))

Epoch 0 started
Train batch 0 started
Train batch 250 started
Train batch 500 started
Train batch 750 started
Train batch 1000 started
Train batch 1250 started
Train batch 1500 started
Train batch 1750 started
Train batch 2000 started
Train batch 2250 started
Train batch 2500 started
Train batch 2750 started
Train batch 3000 started
Train batch 3250 started
Train batch 3500 started
Train batch 3750 started
Train batch 4000 started
Train batch 4250 started
Train batch 4500 started
Train batch 4750 started
Train batch 5000 started
Train batch 5250 started
Train batch 5500 started
Train batch 5750 started
Train batch 6000 started
Train batch 6250 started
Train batch 6500 started
Train batch 6750 started
Train batch 7000 started
Train batch 7250 started
Train batch 7500 started
Train batch 7750 started
Train batch 8000 started
Train batch 8250 started
Train batch 8500 started
Train batch 8750 started


KeyboardInterrupt: ignored

In [201]:
model.fc1.weight

Parameter containing:
tensor([[-0.0442,  0.0485,  0.0601,  ..., -0.0775,  0.0769, -0.0393],
        [ 0.0211, -0.0274,  0.0136,  ...,  0.0097,  0.0018, -0.0133],
        [ 0.0003,  0.0190,  0.0316,  ..., -0.0792,  0.0456, -0.0556],
        ...,
        [ 0.0044,  0.0090, -0.0134,  ...,  0.0054, -0.0088, -0.0224],
        [-0.0001,  0.0661,  0.0640,  ..., -0.0238,  0.0662, -0.0204],
        [-0.0213,  0.0489,  0.0668,  ..., -0.0523,  0.0582, -0.0355]],
       device='cuda:0', requires_grad=True)

In [202]:
post_training_metrics = report_eval_metrics(model, test_data, step=0.01)

Batch 0 started
Batch 25 started
Batch 50 started
Batch 75 started
Batch 100 started
Batch 125 started
Batch 150 started
Batch 175 started
Batch 200 started
Batch 225 started
Batch 250 started
Batch 275 started
Batch 300 started
Batch 325 started


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars


In [209]:
post_training_metrics

{0.0: array([0.04284501, 0.04050879, 1.        , 0.00246572, 0.07786343]),
 0.01: array([0.88115625, 0.1903318 , 0.60673847, 0.89129904, 0.28976523]),
 0.02: array([0.89559813, 0.21099378, 0.59044463, 0.90701218, 0.31089138]),
 0.03: array([0.90160068, 0.22239616, 0.58851146, 0.9133393 , 0.32280542]),
 0.04: array([0.90770383, 0.23577596, 0.58823529, 0.91970132, 0.33662584]),
 0.05: array([0.9126333 , 0.24787618, 0.58823529, 0.92483048, 0.34878009]),
 0.06: array([0.91471239, 0.25336029, 0.58823529, 0.9269938 , 0.35417359]),
 0.07: array([0.91661264, 0.25858929, 0.58823529, 0.92897103, 0.35925114]),
 0.08: array([0.91925063, 0.26621672, 0.58823529, 0.93171588, 0.36654621]),
 0.09: array([0.9199772 , 0.26839718, 0.58823529, 0.93247188, 0.36860777]),
 0.1: array([0.92036842, 0.26958613, 0.58823529, 0.93287896, 0.36972748]),
 0.11: array([0.92061434, 0.27028056, 0.58795913, 0.93314647, 0.37032527]),
 0.12: array([0.92056963, 0.26832076, 0.57939796, 0.9334605 , 0.36678322]),
 0.13: array([

# Saving objects

In [203]:
model_path = "/content/drive/MyDrive/model.pt"

In [204]:
torch.save(model.state_dict(), model_path)

In [205]:
pre_training_metrics_path = "/content/drive/MyDrive/pre_training_metrics.pkl"
post_training_metrics_path = "/content/drive/MyDrive/post_training_metrics.pkl"

In [206]:
import pickle

with open(pre_training_metrics_path, "wb") as f:
    pickle.dump(pre_training_metrics, f)

with open(post_training_metrics_path, "wb") as f:
    pickle.dump(post_training_metrics, f)

In [207]:
train_data_path = "/content/drive/MyDrive/train_data.zip"
test_data_path = "/content/drive/MyDrive/test_data.zip"
eval_data_path = "/content/drive/MyDrive/eval_data.zip"

In [208]:
train_data.to_csv(path_or_buf=train_data_path, index=False)
test_data.to_csv(path_or_buf=test_data_path, index=False)
eval_data.to_csv(path_or_buf=eval_data_path, index=False)

In [163]:
test_read = pd.read_csv(train_data_path)

In [164]:
train_data.shape

(715692, 29)

In [165]:
test_read.shape

(715692, 29)

In [ ]:
train_data.iloc[8169]

tweetid                                                   1217514045751930882
userid                            vzE2Thz2OP5EUbQsVtqv90Ri7KiFTUHafGgb+aJxag=
user_display_name                 vze2thz2op5eubqsvtqv90ri7kiftuhafggb+ajxag=
user_screen_name                  vzE2Thz2OP5EUbQsVtqv90Ri7KiFTUHafGgb+aJxag=
user_reported_location                                                       
user_profile_description                                                  nan
follower_count                                                            528
following_count                                                          1106
account_creation_date                                              2016-02-20
account_language                                                           tr
tweet_language                                                             tr
tweet_text                  : hiçbir esnaf ticari itibarını i̇smini çocuğu...
tweet_time                                                   202

In [ ]:
test_read['tweetid'].iloc[8169]

1217514045751930882

In [ ]:
i = 0
j = 0
bad_j_vals = []

while (i < train_data.shape[0]):
  if str(train_data['tweetid'].iat[i]) != test_read['tweetid'].iat[j]:
    bad_j_vals.append(j)
    j += 1
  else:
    i += 1
    j += 1

bad_j_vals

IndexError: ignored

# Debugging

In [ ]:
times

In [ ]:
torch.cuda.memory_summary(device="cuda", abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 8            |        cudaMalloc retries: 13        |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |   10661 MB |   10674 MB |   10036 GB |   10026 GB |\n|       from large pool |   10658 MB |   10672 MB |   10032 GB |   10021 GB |\n|       from small pool |       2 MB |       3 MB |       4 GB |       4 GB |\n|---------------------------------------------------------------------------|\n| Active memory         |   10661 MB |   10674 MB |   10036 GB |   10026 GB |\n|       from large pool |   10658 MB |   10672 MB |

In [ ]:
for obj in gc.get_objects():
    try:
        if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
            print(type(obj), obj.size())
    except:
        pass

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: astor.all_symbols is deprecated.  Please use astor.symbol_data.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: astor.treewalk is deprecated.  Please use astor.tree_walk.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: astor.codegen is deprecated.  Please use astor.code_gen.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py:151: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn("torch.distributed.reduce_op is deprecated, please use "


<class 'torch.nn.parameter.Parameter'> torch.Size([32000, 768])
<class 'torch.nn.parameter.Parameter'> torch.Size([512, 768])
<class 'torch.nn.parameter.Parameter'> torch.Size([2, 768])
<class 'torch.nn.parameter.Parameter'> torch.Size([768])
<class 'torch.nn.parameter.Parameter'> torch.Size([768])
<class 'torch.nn.parameter.Parameter'> torch.Size([768, 768])
<class 'torch.nn.parameter.Parameter'> torch.Size([768])
<class 'torch.nn.parameter.Parameter'> torch.Size([3072, 768])
<class 'torch.nn.parameter.Parameter'> torch.Size([3072])
<class 'torch.nn.parameter.Parameter'> torch.Size([768, 3072])
<class 'torch.nn.parameter.Parameter'> torch.Size([768])
<class 'torch.nn.parameter.Parameter'> torch.Size([768])
<class 'torch.nn.parameter.Parameter'> torch.Size([768])
<class 'torch.nn.parameter.Parameter'> torch.Size([3072, 768])
<class 'torch.nn.parameter.Parameter'> torch.Size([3072])
<class 'torch.nn.parameter.Parameter'> torch.Size([768, 3072])
<class 'torch.nn.parameter.Parameter'> tor